In [ ]:
import numpy as np
import igraph
from graspologic.cluster import GaussianCluster
from graspologic.embed.ase import AdjacencySpectralEmbed

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt


In [ ]:
# load files
subject = "Yuri" 
weight_file = "/home/bizon/Projects/mturk2/mturk2_code/saved_data/out_models/Yuri_bayes_dyno_Elastic_coordhist.npy"
prob_w = np.load(weight_file)


In [ ]:
last_w = prob_w[-1]
last_w[np.eye(1296).astype(bool)] = 0
last_w.shape
#last_w[last_w < .01] = 0.

In [ ]:
# get spectral embedding and show
SE = AdjacencySpectralEmbed()
emb = SE.fit_transform(last_w)
print("ndims opt.", emb.shape[1])
# 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(emb[:, 0], emb[:, 1], emb[:, 2], s=.5)
plt.show()

In [ ]:
from neurotools.embed import MDScale
import torch

weights = torch.ones(last_w.shape, dtype=torch.float)
weights[torch.from_numpy(last_w) < .001] = 0.
mds = MDScale(1296, embed_dims=2, initialization="xavier", device='cpu', struct="toroid", weights=weights)
target_dists = torch.logit(1 - torch.from_numpy(last_w)) - .26
target_dists = torch.nan_to_num(target_dists, posinf=0, neginf=0)
emb = mds.fit_transform(target_dists)

In [ ]:
# convert embeddings to square
nemb = emb.numpy()
mod_emb = np.mod(nemb, 2 * np.pi)
plt.scatter(mod_emb[:, 0], mod_emb[:, 1])
plt.show()

In [ ]:
clust = GaussianCluster(min_components=2, max_components=20, max_iter=10000, n_init=3)
clust = clust.fit(emb)

In [ ]:
plt.plot(clust.bic_.full)

In [ ]:
labels = clust.predict(emb)

In [ ]:
SE = AdjacencySpectralEmbed(n_components=2)
emb = SE.fit_transform(last_w)
fig = plt.figure()
ax = fig.add_subplot()
ax.scatter(emb[:, 0], emb[:, 1], s=10, c=plt.get_cmap("tab20")(labels))
plt.show()
fig.savefig("SpectralEmbed.svg")

In [ ]:
# get cluster centers 
map = labels.reshape(36, 36)
plt.imshow(map, cmap="tab20")
plt.savefig("cat_map_spectral.svg")
plt.show()

In [ ]:
plt.scatter(mod_emb[:, 0], mod_emb[:, 1], c=labels, cmap="tab20")
plt.show()

In [ ]:
prob_file = "/home/bizon/Projects/mturk2/mturk2_code/saved_data/out_models/Yuri_bayes_dyno_Elastic_probhist.npy"
prob_his = np.load(prob_file)

In [ ]:
from scipy.ndimage import gaussian_filter1d
plt.plot(gaussian_filter1d(np.exp(prob_his.squeeze()), mode="constant", cval=.25, sigma=50))
plt.savefig("y_choice_probhist.svg")